补充2类的实验结果

In [1]:
# main_process.py
# 主程序入口，进行数据加载、预处理、模型训练与测试，并评估分类准确率

import os
import pandas as pd
import numpy as np
import random
import torch
import torch.nn.functional as F
from torch.distributions import Categorical
from torch import nn
from data_loader import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from collections import Counter
from autoencoder import *
from evt import *
import argparse
import time
from datetime import timedelta
from sklearn.cluster import KMeans, DBSCAN
import warnings

start_time = time.time()

In [ ]:
# 设置随机种子，保证实验可复现
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# 解析命令行参数，支持自定义数据集名称
default_data_name = 'demo'
# parser = argparse.ArgumentParser()
# parser.add_argument('--data_name', type = str, default = default_data_name, help = 'data name')
# args = parser.parse_args()
# data_name = args.data_name
data_name = default_data_name

# 加载数据集，并进行标准化
# data_load: 特征数据, label_load: 标签
data_load, label_load = read_dataset2024(data_name)
data_load = StandardScaler().fit_transform(data_load)

# 统计每个类别的样本数，确定训练/测试划分
count_number = Counter(label_load)
min_num = np.array(list(count_number.values())).min()  # 最小类别样本数
test_per_class = 300  # 每类测试样本数
num_per_class = min_num - test_per_class  # 每类训练样本数
dim = data_load.shape[1]  # 特征维度
b_size = test_per_class   # 批量大小
loss_func = nn.MSELoss()  # 损失函数

sum_num = len(set(list((label_load))))  # 类别总数
train_num = 2  # 训练类别数（只用一个类别做已知，其余为新类）
newclass_num = sum_num - train_num  # 新类别数

# 打乱数据顺序
shun = list(range(data_load.shape[0]))
random.shuffle(shun)
data_load = data_load[shun]
label_load = label_load[shun]

# 按数字顺序排列类别索引，而不是随机排列
# allIndex = np.arange(train_num + newclass_num)
allIndex = np.random.permutation(train_num + newclass_num)


# 计算数据加载部分耗费的时间
data_loading_end = time.time()
data_loading_time = data_loading_end - start_time
print(f"=== Data Loading Time ===")
print(f"Data loading time: {str(timedelta(seconds=data_loading_time))}")

d:\Study\Github\Trident\code\data_loader.py:76: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace([np.inf, -np.inf], np.nan)


Class distribution after oversampling:
Label
11    220202
8     195419
9     125020
10    118008
1      19138
2      18546
5      10097
3       7242
0       1645
4        600
6        600
7        600
Name: count, dtype: int64
=== Data Loading Time ===
Data loading time: 0:00:05.332618


In [3]:
# 构建训练集（只包含已知类别）
data = np.zeros((num_per_class * (train_num), dim))
label = np.zeros(num_per_class * (train_num))
for pos in range(train_num):
    i = allIndex[pos]
    data[pos * num_per_class:(pos + 1) * num_per_class,:] = data_load[label_load==i][0:num_per_class, :]
    label[pos * num_per_class:(pos + 1) * num_per_class] = i


# 构建流式测试集（包含所有类别，已知类标记为原标签，未知类标记为999）
streamdata = np.zeros((test_per_class * (train_num + newclass_num), dim))
streamlabel = np.zeros(test_per_class * (train_num + newclass_num))
gtlabel = np.zeros(test_per_class * (train_num + newclass_num))
for pos in range(train_num + newclass_num):
    i = allIndex[pos]
    streamdata[pos * test_per_class:(pos + 1) * test_per_class,:] = data_load[label_load==i][-test_per_class:, :]
    gtlabel[pos * test_per_class:(pos + 1) * test_per_class] = i
    if pos < train_num:
        streamlabel[pos * test_per_class:(pos+1) * test_per_class] = i
    else:
        streamlabel[pos * test_per_class:(pos + 1) * test_per_class] = 999
        
# 输出 gtlabel 中按出现顺序的不同标签
unique_labels = pd.Series(gtlabel).unique()
print("Distinct labels in 'gtlabel':", unique_labels)


# 根据标签统计，筛选样本数大于50的类别
# 返回当前存在的类别列表
def make_lab(label):
    xianyou = pd.DataFrame(label).value_counts()
    curr_lab = []
    for j1 in xianyou.keys():
        for i in range(train_num):
            if (xianyou[j1] > 50) and (j1[0] == allIndex[i]):
                curr_lab.append(j1[0])
                break
        # if xianyou[j1] > 50:
        #     curr_lab.append(j1[0])
    return curr_lab

# 针对每个类别训练自编码器模型，并用SPOT方法确定阈值
# 返回模型列表、阈值列表、类别列表
def train(data, label, curr_lab):
    mod_ls = []
    thred_ls = []
    class_ls = []
    batch = 10
    epoch = 10
    y_in, y1, y2, y3, y4 = data_load.shape[1], 256, 128, 64, 32
    for i in curr_lab:
        class_ls.append(i)
        model = Autoencoder(y_in, y1, y2, y3, y4)
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay = 5e-4)
        # 训练自编码器
        for i2 in range(epoch):
            shun = list(range(data[label==i].shape[0]))
            random.shuffle(shun)
            for i3 in range(int(data[label==i].shape[0] / batch)):
                data_input = torch.from_numpy(data[label==i][shun][i3 * batch : (i3+1) * batch]).float()
                pred = model(data_input)
                loss = loss_func(pred, data_input)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        mod_eva = model.eval()
        mod_ls.append(model)
        # 计算重构误差，用于阈值确定
        mse_ls = []
        for i4 in range(int(data[label==i].shape[0] / batch)):
            data_input = torch.from_numpy(data[label==i][i4 * batch : (i4+1) * batch]).float()
            pred = model(data_input)
            for i5 in range(pred.shape[0]):
                loss = loss_func(pred[i5], data_input[i5])
                mse_ls.append(float(loss.detach().numpy()))
        data_input = torch.from_numpy(data[label==i][(int(data[label==i].shape[0] / batch)) * batch:]).float()
        pred = model(data_input)
        for i5 in range(pred.shape[0]):
            loss = loss_func(pred[i5], data_input[i5])
            mse_ls.append(float(loss.detach().numpy()))
        loss_list_use = np.array(mse_ls)
        q = 5e-2 # 风险参数，可调
        if len(loss_list_use) == 0:
            thred_ls.append(0)
            continue
        # if len(loss_list_use) < 50 or np.all(loss_list_use == loss_list_use[0]):
        #     thred_ls.append(np.max(loss_list_use) + 1e-3)
        #     continue
        try:
            s = SPOT(q)
            s.fit(loss_list_use, loss_list_use)
            s.initialize()
            results = s.run_simp()
            # 阈值选取
            if results['thresholds'][0] > 0:
                thred_ls.append(results['thresholds'][0])
            else:
                thred_ls.append(np.sort(s.init_data)[int(0.85 * s.init_data.size)])
        except Exception as e:
            thred_ls.append(np.max(loss_list_use) + 1e-3)
    return mod_ls, thred_ls, class_ls

# 计算数据预处理部分耗费的时间
preprocessing_end = time.time()
preprocessing_time = preprocessing_end - data_loading_end

print(f"\n=== Data Preprocessing Time ===")
print(f"Preprocessing time: {str(timedelta(seconds=preprocessing_time))}")

Distinct labels in 'gtlabel': [11.  7. 10.  1.  3.  9.  6.  2.  4.  0.  8.  5.]

=== Data Preprocessing Time ===
Preprocessing time: 0:00:00.151524


In [4]:
# 主流程入口
if __name__ == '__main__':
    print('=== Initializing ===')
    # 获取当前类别
    curr_num = train_num
    curr_lab = allIndex[:curr_num]
    # 训练初始模型
    mod_ls, thred_ls, class_ls = train(data, label, curr_lab)
    
    res_ls = []  # 预测结果列表
    # 先预测已知的train_num个类别
    for i5 in range(test_per_class * train_num):
        # 对当前样本用所有模型计算重构误差
        data_input = torch.from_numpy(streamdata[i5]).float()
        mse_test = []
        for model in mod_ls:
            mod_eva = model.eval()
            pred = model(data_input)
            loss = loss_func(pred, data_input)
            mse_test.append(float(loss.detach().numpy()))
        # 判断是否为新类
        cand_res = np.array(mse_test)[np.array(mse_test) < np.array(thred_ls)]
        if len(cand_res) == 0:
            res_ls.append(999)
        else:
            min_loss_res = cand_res.min()
            res_ls.append(class_ls[mse_test.index(min_loss_res)])
    # Calculate accuracy for the first set of predictions
    initial_preds = np.array(res_ls)
    initial_true = gtlabel[:len(initial_preds)]
    initial_accuracy = accuracy_score(initial_true, initial_preds)
    print(f"Initial prediction accuracy (first {train_num} classes): {initial_accuracy:.4f}")

    # Calculate other metrics for the initial prediction
    initial_f1 = f1_score(initial_true, initial_preds, average='macro', zero_division=0)
    initial_recall = recall_score(initial_true, initial_preds, average='macro', zero_division=0)
    print(f"Initial F1 Score: {initial_f1:.4f}")
    print(f"Initial Recall: {initial_recall:.4f}")

=== Initializing ===
Initial prediction accuracy (first 2 classes): 0.8900
Initial F1 Score: 0.6233
Initial Recall: 0.5933
